<a href="https://colab.research.google.com/github/IgorDiamandi/base_ML_Project/blob/BestResult/Colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import gdown
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pathlib import Path

def download_from_gdrive(url, filename):
    # Extract the file ID from the URL
    file_id = url.split('/')[-2]
    download_url = f"https://drive.google.com/uc?id={file_id}"

    # Download the file
    if Path(filename).exists():
        print(f"File '{filename}' already exists. Skipping download.")
    else:
        gdown.download(download_url, filename, quiet=False)
        print(f"File downloaded as: {filename}")

train = 'https://drive.google.com/file/d/1guqSpDv1Q7ZZjSbXMYGbrTvGns0VCyU5/view?usp=drive_link'
valid = 'https://drive.google.com/file/d/1j7x8xhMimKbvW62D-XeDfuRyj9ia636q/view?usp=drive_link'
# Example usage

download_from_gdrive(train, 'train.csv')
download_from_gdrive(valid, 'valid.csv')

File 'train.csv' already exists. Skipping download.
File 'valid.csv' already exists. Skipping download.


In [118]:
# model functions

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def train_and_evaluate_model(X_train, X_test, y_train, y_test, tree_depth, level_of_parallelism, number_of_trees,
                             max_features):
    for depth in tree_depth:
        model = RandomForestRegressor(
                random_state=100,
                n_jobs=level_of_parallelism,
                n_estimators=number_of_trees,
                max_depth=depth,
                max_features=max_features)


        print('Fitting the model...')
        model.fit(X_train, y_train)

        print('Testing the model...')
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        rmse_test = get_rmse(y_test, y_test_pred)
        rmse_train = get_rmse(y_train, y_train_pred)

        print(f'Tree depth - {depth}')
        print(f'STD Test - {y_test.std()}')
        print(f'STD Train - {y_train.std()}')
        print(f'RMSE Test - {rmse_test}')
        print(f'RMSE Train - {rmse_train}')

    return model


In [119]:
# data functions
import zipfile
from datetime import datetime
import numpy as np
from scipy.stats import zscore
import pandas as pd


#Function to retrieve statistic parameters from numerical columns of the train dataframe
def compute_statistics(df):
    numeric_df = df.select_dtypes(include='number')

    mean_values = numeric_df.mean()
    iqr_values = numeric_df.quantile(0.75) - numeric_df.quantile(0.25)
    zscore_values = (numeric_df.mean() / numeric_df.std()).mean()

    # Mean without extremes (assuming extremes are values outside 1.5*IQR)
    def mean_without_extremes(series):
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        filtered_series = series[(series >= Q1 - 1.5 * IQR) & (series <= Q3 + 1.5 * IQR)]
        return filtered_series.mean()

    mean_no_extremes_values = numeric_df.apply(mean_without_extremes)

    statistics_df = pd.DataFrame({
        'mean': mean_values,
        'iqr': iqr_values,
        'zscore': [zscore_values] * len(numeric_df.columns),
        'mean_without_extremes': mean_no_extremes_values
    })

    statistics_df = statistics_df.T
    return statistics_df


def get_stat_value(method, column, statistics_df):
  if method not in statistics_df.index:
    raise ValueError(f"Method {method} not found in statistics DataFrame")
  return statistics_df.loc[method, column]


def replace_outliers(df, column, method, statistics_df):
    stat_value = get_stat_value(method, column, statistics_df)

    # Define the threshold for identifying outliers
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace outliers with the statistical method value
    df[column] = df[column].apply(lambda x: stat_value if x < lower_bound or x > upper_bound else x)

    return df


def replace_nans(df, column, method, statistics_df):
    stat_value = get_stat_value(method, column, statistics_df)
    df[column] = df[column].fillna(stat_value)

    return df


def get_rmse(y, y_pred):
    return mean_squared_error(y, y_pred) ** 0.5


def split_product_class_series(series):
    equipment_type = []
    details = []

    for item in series:
        if pd.isna(item):
            equipment_type.append(None)
            details.append(None)
        else:
            split_item = item.split(' - ', 1)
            equipment_type.append(split_item[0])
            details.append(split_item[1] if len(split_item) > 1 else None)

    return pd.Series(equipment_type), pd.Series(details)

In [136]:
df = pd.read_csv('train.csv')
df_valid = pd.read_csv('valid.csv')

<ipython-input-136-6adbd7aebcbe>:1: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


In [137]:
# convert 'YearMade' column to 'Age'

df['Age'] = 2024 - df['YearMade']
# Clamp 'YearMade' to a reasonable minimum before converting to datetime
#df['AgeAtLastSale'] = pd.to_datetime(df['saledate']) - pd.to_datetime(df['YearMade'].clip(lower=1900), format='%Y')
df.drop('YearMade', axis=1, inplace=True)

df_valid['Age'] = 2024 - df_valid['YearMade']
# Clamp 'YearMade' to a reasonable minimum before converting to datetime, and convert 'saledate' to datetime
#df_valid['AgeAtLastSale'] = pd.to_datetime(df_valid['saledate']) - pd.to_datetime(df_valid['YearMade'].clip(lower=1900), format='%Y')
df_valid.drop('YearMade', axis=1, inplace=True)

In [138]:
# split 'fiProductDesc' column

df['ProductClassName'],df['ProductClassCharacteristic'] = split_product_class_series(df['fiProductClassDesc'])
df_valid['ProductClassName'],df_valid['ProductClassCharacteristic'] = split_product_class_series(df_valid['fiProductClassDesc'])

In [139]:
# convert 'MachineID' column into 'TimesAppearing' column

df['TimesAppearing'] = df['MachineID'].map(df['MachineID'].value_counts())
df.drop('MachineID', axis=1, inplace=True)

df_valid['TimesAppearing'] = df_valid['MachineID'].map(df_valid['MachineID'].value_counts())
df_valid.drop('MachineID', axis=1, inplace=True)

In [140]:
# remove duplicated columns: 'ProductGroupDesc'

df.drop(['ProductGroupDesc','ProductClassName'], axis=1, inplace=True)
df_valid.drop(['ProductGroupDesc','ProductClassName'], axis=1, inplace=True)

In [141]:
# replace NaN values with 'missing' for non-numeric fields
for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna('missing', inplace=True)


for col in df_valid.columns:
    if df_valid[col].dtype == 'object':
        df_valid[col].fillna('missing', inplace=True)

In [142]:
# replace NaN with -1 in 'ID' columns

df[['datasource', 'auctioneerID']] = df[['datasource', 'auctioneerID']].fillna(-1)
df_valid[['datasource', 'auctioneerID']] = df_valid[['datasource', 'auctioneerID']].fillna(-1)

In [143]:
#Split train and test data

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['SalePrice']), df['SalePrice'], test_size=0.3, random_state=100)

In [144]:
# get statistics dataframe using compute_statistics function from the X_train
statistics_df = compute_statistics(X_train)

In [145]:
""" Use replace_outliers function in order to replace outliers
in the 'MachineHoursCurrentMeter' column using statistics_df and IQR method """

X_train = replace_outliers(X_train, 'MachineHoursCurrentMeter', 'iqr', statistics_df)
X_test = replace_outliers(X_test, 'MachineHoursCurrentMeter', 'iqr', statistics_df)
df_valid = replace_outliers(df_valid, 'MachineHoursCurrentMeter', 'iqr', statistics_df)

In [146]:
""" Use replace_outliers function in order to replace outliers
in the 'Age' column using statistics_df and mean method """

X_train = replace_outliers(X_train, 'Age', 'mean', statistics_df)
X_test = replace_outliers(X_test, 'Age', 'mean', statistics_df)
df_valid = replace_outliers(df_valid, 'Age', 'mean', statistics_df)

In [147]:
""" Use replace_nans function in order to replace outliers
in the 'MachineHoursCurrentMeter' column using statistics_df and IQR method """

X_train = replace_nans(X_train, 'MachineHoursCurrentMeter', 'iqr', statistics_df)
X_test = replace_nans(X_test, 'MachineHoursCurrentMeter', 'iqr', statistics_df)
df_valid = replace_nans(df_valid, 'MachineHoursCurrentMeter', 'iqr', statistics_df)

In [148]:
# replace NaN with 'Missing' in textual columns

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        if X_train[col].apply(type).nunique() > 1:
            X_train[col] = X_train[col].fillna('Missing').astype(str)
        X_train[col] = le.fit_transform(X_train[col])

for col in X_test.columns:
    if X_test[col].dtype == 'object':
        if X_test[col].apply(type).nunique() > 1:
            X_test[col] = X_test[col].fillna('Missing').astype(str)
        X_test[col] = le.fit_transform(X_test[col])


for col in df_valid.columns:
    if df_valid[col].dtype == 'object':
        if df_valid[col].apply(type).nunique() > 1:
            df_valid[col] = df_valid[col].fillna('Missing').astype(str)
        df_valid[col] = le.fit_transform(df_valid[col])



In [149]:
# Check NaN in X_Train

X_train.isna().sum()

SalesID                       0
ModelID                       0
datasource                    0
auctioneerID                  0
MachineHoursCurrentMeter      0
UsageBand                     0
saledate                      0
fiModelDesc                   0
fiBaseModel                   0
fiSecondaryDesc               0
fiModelSeries                 0
fiModelDescriptor             0
ProductSize                   0
fiProductClassDesc            0
state                         0
ProductGroup                  0
Drive_System                  0
Enclosure                     0
Forks                         0
Pad_Type                      0
Ride_Control                  0
Stick                         0
Transmission                  0
Turbocharged                  0
Blade_Extension               0
Blade_Width                   0
Enclosure_Type                0
Engine_Horsepower             0
Hydraulics                    0
Pushblock                     0
Ripper                        0
Scarifie

In [150]:
model = train_and_evaluate_model(X_train, X_test, y_train, y_test, [15], -1, 20,
                                 0.5)

Fitting the model...
Testing the model...
Tree depth - 15
STD Test - 22995.709788960918
STD Train - 23054.44179044156
RMSE Test - 12747.583073222002
RMSE Train - 8734.391293253819


In [23]:
X_valid = df_valid
y_valid_pred = model.predict(X_valid)

# Create the prediction DataFrame with only 'SalesID' and 'Predicted_SalePrice'
df_predictions = pd.DataFrame({
    'SalesID': df_valid['SalesID'],
    'SalePrice': y_valid_pred
})
df_predictions.to_csv('valid_predictions.csv', index=False)

In [24]:
from google.colab import files
files.download('valid_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# print features importance order it descending

feature_importances = model.feature_importances_
feature_names = X_train.columns

importance_list = [(importance, feature) for feature, importance in zip(feature_names, feature_importances)]
importance_list.sort(reverse=True)

for importance, feature in importance_list:
    print(f"{feature}: {importance}")


Age: 0.19937407506585206
ProductSize: 0.10691889839352231
SalesID: 0.08207153754382275
ModelID: 0.07300923476058667
ProductClassCharacteristic: 0.05319805945419628
fiSecondaryDesc: 0.05011915152936151
fiModelDesc: 0.04547696431017013
fiProductClassDesc: 0.04328313362428725
fiBaseModel: 0.03834234202871468
fiModelDescriptor: 0.03582192624646638
saledate: 0.030655716543375417
state: 0.020760749250534297
ProductGroup: 0.020564596106261848
Tire_Size: 0.016754501107996818
Enclosure: 0.015835870041758633
auctioneerID: 0.015216829140987745
MachineHoursCurrentMeter: 0.014636762750531834
Drive_System: 0.013355963856597172
Pad_Type: 0.012103941017270674
Coupler_System: 0.011823749254768676
Track_Type: 0.011386670539874175
datasource: 0.009893314497543201
Ripper: 0.007958961846226305
fiModelSeries: 0.007183775188782225
Hydraulics: 0.007051763375061887
Forks: 0.007007770072411807
TimesAppearing: 0.005472211436897077
Grouser_Tracks: 0.005382902937514509
Ride_Control: 0.005382876577428912
Coupler: 0